<a href="https://colab.research.google.com/github/vvivvi/kaggle-c1/blob/master/Kaggle_C1_mean_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def write_predictions_by_array(array, filename):
  df=pd.DataFrame(array)
  df.columns=['item_cnt_month']
  df.to_csv(os.path.join(DATA_FOLDER, filename), index_label='ID')

In [2]:
!pip install tqdm
!pip install lightgbm
!pip install catboost

In [3]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm 

for p in [np, pd, scipy, sklearn, lightgbm]:
    print (p.__name__, p.__version__)

numpy 1.18.1
pandas 0.25.3
scipy 1.4.1
sklearn 0.22.1
lightgbm 2.3.1


In [4]:
def clipped_rmse(gt, predicted,clip_min=0, clip_max=20):
  target=np.minimum(np.maximum(gt,clip_min), clip_max)
  return np.sqrt((target-predicted)**2).mean()

In [5]:
from itertools import product
import gc
from tqdm import tqdm_notebook


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

!wget -O competitive-data-science-predict-future-sales.zip https://github.com/vvivvi/kaggle-c1/blob/master/competitive-data-science-predict-future-sales.zip?raw=true
!mkdir competitive-data-science-predict-future-sales
!unzip -o competitive-data-science-predict-future-sales.zip -d competitive-data-science-predict-future-sales

DATA_FOLDER = 'competitive-data-science-predict-future-sales'

sales    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_cats = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
sample_submission = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

monthly_summary_train=sales[['date_block_num','shop_id','item_id','item_cnt_day']].groupby(['date_block_num','shop_id','item_id'], as_index=False).sum()
monthly_summary_dec14 = monthly_summary_train[monthly_summary_train['date_block_num'] == 23]
monthly_summary_oct15 = monthly_summary_train[monthly_summary_train['date_block_num'] == 33]

--2020-03-13 15:49:00--  https://github.com/vvivvi/kaggle-c1/blob/master/competitive-data-science-predict-future-sales.zip?raw=true
Selvitetään osoitetta github.com (github.com)... 140.82.118.4
Yhdistetään palvelimeen github.com (github.com)|140.82.118.4|:443... yhdistetty.
HTTP-pyyntö lähetetty, odotetaan vastausta... 302 Found
Sijainti: https://github.com/vvivvi/kaggle-c1/raw/master/competitive-data-science-predict-future-sales.zip [seurataan]
--2020-03-13 15:49:01--  https://github.com/vvivvi/kaggle-c1/raw/master/competitive-data-science-predict-future-sales.zip
Käytetään uudelleen yhteyttä github.com:443.
HTTP-pyyntö lähetetty, odotetaan vastausta... 302 Found
Sijainti: https://raw.githubusercontent.com/vvivvi/kaggle-c1/master/competitive-data-science-predict-future-sales.zip [seurataan]
--2020-03-13 15:49:01--  https://raw.githubusercontent.com/vvivvi/kaggle-c1/master/competitive-data-science-predict-future-sales.zip
Selvitetään osoitetta raw.githubusercontent.com (raw.githubuserc

In [7]:
test_shops=test_spec['shop_id'].unique()
test_items=test_spec['item_id'].unique()
date_block_val = 33
date_block_test = 35 # Dec 2015
sales = sales[sales['shop_id'].isin(test_shops)]

In [8]:
shop_and_item_id_to_test_id={}

sr = test_spec['item_id'].astype(str) + '_' + test_spec['shop_id'].astype(str)
for index, val in sr.iteritems():
  shop_and_item_id_to_test_id[val] = index



In [9]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# add test month     
grid.append(np.array(list(product(*[test_shops, test_items, [date_block_test]])),dtype='int32'))

print('Grid tail: ', grid[-1])    

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
del grid, gb 
gc.collect();

Grid tail:  [[    5  5037    35]
 [    5  5320    35]
 [    5  5233    35]
 ...
 [   45 15757    35]
 [   45 19648    35]
 [   45   969    35]]


/home/vvi/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


In [10]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [2, 3, 4, 5,6,7,8,9,10,11, 12] 
# there's one month gap between and test periods

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col.split('_')[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

/home/vvi/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [11]:
fit_cols


['target_lag_2',
 'target_item_lag_2',
 'target_shop_lag_2',
 'target_lag_3',
 'target_item_lag_3',
 'target_shop_lag_3',
 'target_lag_4',
 'target_item_lag_4',
 'target_shop_lag_4',
 'target_lag_5',
 'target_item_lag_5',
 'target_shop_lag_5',
 'target_lag_6',
 'target_item_lag_6',
 'target_shop_lag_6',
 'target_lag_7',
 'target_item_lag_7',
 'target_shop_lag_7',
 'target_lag_8',
 'target_item_lag_8',
 'target_shop_lag_8',
 'target_lag_9',
 'target_item_lag_9',
 'target_shop_lag_9',
 'target_lag_10',
 'target_item_lag_10',
 'target_shop_lag_10',
 'target_lag_11',
 'target_item_lag_11',
 'target_shop_lag_11',
 'target_lag_12',
 'target_item_lag_12',
 'target_shop_lag_12']

In [12]:
dates = all_data['date_block_num']

dates_train = dates[dates <  date_block_val]
dates_val  = dates[dates == date_block_val]
dates_test  = dates[dates == date_block_test]

X_train = all_data.loc[dates <  date_block_val] #.drop(to_drop_cols, axis=1)
X_val =  all_data.loc[dates == date_block_val] #.drop(to_drop_cols, axis=1)
X_trainval =  all_data.loc[dates < date_block_test] #.drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == date_block_test]# .drop(to_drop_cols, axis=1)


y_train = np.clip(all_data.loc[dates <  date_block_val, 'target'].values,0,20)
y_trainval = np.clip(all_data.loc[dates <  date_block_test, 'target'].values,0,20)
y_val =  np.clip(all_data.loc[dates == date_block_val, 'target'].values,0,20)

In [13]:
from sklearn.model_selection import KFold

def mean_encode_kfold(all_data, coded_columns, target_column):
    y_tr = all_data[target_column].values
    kf = KFold(5,shuffle=True, random_state=123) 
    
    target_mean=all_data[target_column].mean()
    print('global target mean: ', target_mean)
    
    for col in coded_columns:
        all_data[col+'_target_enc']=target_mean

    for train_index, fill_index in kf.split(y_tr):
        for col in coded_columns:
            X_tr, X_fill = all_data.iloc[train_index,:], all_data.iloc[fill_index,:]
            encoded = X_fill[col].map(X_tr.groupby(col)[target_column].mean())
            X_fill[col + '_target_enc'] = encoded
            all_data.iloc[fill_index] = X_fill
    
    for col in coded_columns:
        all_data[col+'_target_enc'].fillna(target_mean, inplace=True)
        
def mean_encode_simple(train_data, coded_data, coded_columns, target_column):
    target_mean=train_data[target_column].mean()
    for col in coded_columns:
        col_target_mean_train = train_data.groupby(col)[target_column].mean()
        coded_data[col+'_target_enc'] = coded_data[col].map(col_target_mean_train)
        coded_data[col+'_target_enc'].fillna(train_data[target_column].mean(), inplace=True)
        

In [14]:
encoded_columns = ['item_id','shop_id','item_category_id']

# add mean encodings to training parts of data
# using 5-fold regularization scheme
mean_encode_kfold(X_train, encoded_columns,'target')    
mean_encode_kfold(X_trainval, encoded_columns,'target')    

# add unregularized mean encodings to predicted part of data
mean_encode_simple(X_train, X_val, encoded_columns,'target')    
mean_encode_simple(X_trainval,X_test, encoded_columns,'target')


global target mean:  0.3380356


/home/vvi/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/vvi/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/vvi/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

global target mean:  0.33618283


/home/vvi/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
import re

lagged_columns = [col for col in X_train.columns if re.search('lag',col)]
id_columns = [col for col in X_train.columns if re.search('id$',col)]
target_enc_columns = [col for col in X_train.columns if re.search('target_enc$',col)]


In [16]:
# create versions of data with different sets of variables included

# version of data including just lagged sales data
X_train_lagged = X_train[lagged_columns]
X_trainval_lagged = X_trainval[lagged_columns]
X_val_lagged = X_val[lagged_columns]
X_test_lagged = X_test[lagged_columns]

# version of data including lagged sales data and {shop, item, category} ids as such (as in the code borrowed from the 
# ensembling exercise notebook)

X_train_id_and_lagged = X_train[lagged_columns+id_columns]
X_trainval_id_and_lagged = X_trainval[lagged_columns+id_columns]
X_val_id_and_lagged = X_val[lagged_columns+id_columns]
X_test_id_and_lagged = X_test[lagged_columns+id_columns]

# version of data including lagged sales data and target encodings of {shop, item, category} ids 

X_train_enc_and_lagged = X_train[lagged_columns+target_enc_columns]
X_trainval_enc_and_lagged = X_trainval[lagged_columns+target_enc_columns]
X_val_enc_and_lagged = X_val[lagged_columns+target_enc_columns]
X_test_enc_and_lagged = X_test[lagged_columns+target_enc_columns]

# version of data including lagged sales data and target encodings of {shop, item, category} ids 
# as well as ids as such 

X_train_id_and_enc_and_lagged = X_train[lagged_columns+target_enc_columns+id_columns]
X_trainval_id_and_enc_and_lagged = X_trainval[lagged_columns+target_enc_columns+id_columns]
X_val_id_and_enc_and_lagged = X_val[lagged_columns+target_enc_columns+id_columns]
X_test_id_and_enc_and_lagged = X_test[lagged_columns+target_enc_columns+id_columns]




In [17]:
X_train.columns


Index(['shop_id', 'item_id', 'date_block_num', 'target', 'target_shop',
       'target_item', 'target_lag_2', 'target_item_lag_2', 'target_shop_lag_2',
       'target_lag_3', 'target_item_lag_3', 'target_shop_lag_3',
       'target_lag_4', 'target_item_lag_4', 'target_shop_lag_4',
       'target_lag_5', 'target_item_lag_5', 'target_shop_lag_5',
       'target_lag_6', 'target_item_lag_6', 'target_shop_lag_6',
       'target_lag_7', 'target_item_lag_7', 'target_shop_lag_7',
       'target_lag_8', 'target_item_lag_8', 'target_shop_lag_8',
       'target_lag_9', 'target_item_lag_9', 'target_shop_lag_9',
       'target_lag_10', 'target_item_lag_10', 'target_shop_lag_10',
       'target_lag_11', 'target_item_lag_11', 'target_shop_lag_11',
       'target_lag_12', 'target_item_lag_12', 'target_shop_lag_12',
       'item_category_id', 'item_id_target_enc', 'shop_id_target_enc',
       'item_category_id_target_enc'],
      dtype='object')

In [50]:
import lightgbm as lgb
from sklearn.metrics import r2_score

lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train_lagged, label=y_train), 100)
pred_lgb_val = np.clip(model.predict(X_val_lagged),0,20)

print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE of lgb predictions is ', clipped_rmse(y_val, pred_lgb_val))

model = lgb.train(lgb_params, lgb.Dataset(X_trainval_lagged, label=y_trainval), 100)
pred_lgb_test = np.clip(model.predict(X_test_lagged), 0, 20)
write_predictions_by_array(pred_lgb_test, 'submission-lgb-trainval-laggedonly.csv')
# LB 1.007/1.002


Validation R-squared for LightGBM is 0.274248
Clipped RMSE of lgb predictions is  0.36589291964910403


In [51]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train_id_and_lagged, label=y_train), 100)
pred_lgb_val = np.clip(model.predict(X_val_id_and_lagged),0,20)

print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE of lgb predictions is ', clipped_rmse(y_val, pred_lgb_val))

model = lgb.train(lgb_params, lgb.Dataset(X_trainval_id_and_lagged, label=y_trainval), 100)
pred_lgb_test = np.clip(model.predict(X_test_id_and_lagged), 0, 20)
write_predictions_by_array(pred_lgb_test, 'submission-lgb-trainval-id_and_lagged.csv')
# LB 0.9408 / 0.9437


Validation R-squared for LightGBM is 0.356951
Clipped RMSE of lgb predictions is  0.34688036208330486


In [53]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train_enc_and_lagged, label=y_train), 100)
pred_lgb_val = np.clip(model.predict(X_val_enc_and_lagged),0,20)

print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE of lgb predictions is ', clipped_rmse(y_val, pred_lgb_val))

model = lgb.train(lgb_params, lgb.Dataset(X_trainval_enc_and_lagged, label=y_trainval), 100)
pred_lgb_test = np.clip(model.predict(X_test_enc_and_lagged), 0, 20)
write_predictions_by_array(pred_lgb_test, 'submission-lgb-trainval-enc_and_lagged.csv')
# LB 0.9888 / 0.9889

Validation R-squared for LightGBM is 0.217345
Clipped RMSE of lgb predictions is  0.3773479356644351


In [57]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train_id_and_enc_and_lagged, label=y_train), 100)
pred_lgb_val = np.clip(model.predict(X_val_id_and_enc_and_lagged),0,20)

print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE of lgb predictions is ', clipped_rmse(y_val, pred_lgb_val))

model = lgb.train(lgb_params, lgb.Dataset(X_trainval_id_and_enc_and_lagged, label=y_trainval), 100)
pred_lgb_test = np.clip(model.predict(X_test_id_and_enc_and_lagged), 0, 20)
write_predictions_by_array(pred_lgb_test, 'submission-lgb-trainval-id_enc_and_lagged.csv')
# LB 0.9806 / 0.9799

Validation R-squared for LightGBM is 0.246784
Clipped RMSE of lgb predictions is  0.37128303556281833



Try different types of models

In [ ]:
from sklearn.metrics import r2_score


import catboost
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#scaler.fit(X_train_id_and_enc_and_lagged)

reg = catboost.CatBoostRegressor(iterations=6000, task_type='GPU')
reg.fit(X_train_id_and_enc_and_lagged,y_train)
pred_ll_val = np.clip(reg.predict(X_val_id_and_enc_and_lagged), 0, 20)

print('Validation R-squared for catboodtis %f' % r2_score(y_val, pred_ll_val))
print('Clipped RMSE of catboost predictions is ', clipped_rmse(y_val, pred_ll_val))

reg.fit(X_trainval_id_and_enc_and_lagged, y_trainval)
pred_ll_test = np.clip(reg.predict(X_test_id_and_enc_and_lagged), 0, 20)
write_predictions_by_array(pred_ll_test, 'submission-catboost-trainval-id_and_enc_and_lagged.csv')

# LB 1.024 / 1.026


In [ ]:
from sklearn.metrics import r2_score


import catboost
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#scaler.fit(X_train_id_and_enc_and_lagged)

reg = catboost.CatBoostRegressor(iterations=6000, task_type='GPU')
reg.fit(X_train_id_and_lagged,y_train)
pred_ll_val = np.clip(reg.predict(X_val_id_and_lagged), 0, 20)

print('Validation R-squared for catboodtis %f' % r2_score(y_val, pred_ll_val))
print('Clipped RMSE of catboost predictions is ', clipped_rmse(y_val, pred_ll_val))

reg.fit(X_trainval_id_and_lagged, y_trainval)
pred_ll_test = np.clip(reg.predict(X_test_id_and_lagged), 0, 20)
write_predictions_by_array(pred_ll_test, 'submission-catboost-trainval-id_and_lagged.csv')


In [ ]:
import lightgbm as lgb

lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb_val = np.clip(model.predict(X_val),0,20)

print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE of lgb predictions is ', clipped_rmse(y_val, pred_lgb_val))

In [ ]:
print('Clipped RMSE of lr predictions is ', clipped_rmse(y_val, np.clip(pred_lr_val,0,20)))
print('Clipped RMSE of lgb predictions is ', clipped_rmse(y_val, np.clip(pred_lgb_val,0,20)))


In [ ]:
sr = test_spec['item_id'].astype(str) + '_' + test_spec['shop_id'].astype(str)
sr2 = X_test['item_id'].astype(str) + '_' + X_test['shop_id'].astype(str)

equal=True
for i in range(sr.shape[0]):
  if sr.values[i] != sr2.values[i]:
    equal=False
    break

print('Equal: ', equal)


It seems to be that the above code orders X_test in the same order as the submission is expected to be 

In [ ]:
lr = LinearRegression()
lr.fit(X_trainval.values, y_trainval)
pred_lr_test = np.clip(lr.predict(X_test.values),0,20)
write_predictions_by_array(pred_lr_test, 'submission-lr-trainval-all-ext.csv')

# lb scores 1.105/1.102


In [ ]:
# pred_lr_test=np.clip(pred_lr_test,0,20)
print(min(pred_lr_test))
plt.hist(pred_lr_test,bins=21)
plt.show()


In [ ]:
import lightgbm as lgb

lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }

model = lgb.train(lgb_params, lgb.Dataset(X_trainval, label=y_trainval), 100)
pred_lgb_test = np.clip(model.predict(X_test), 0, 20)
write_predictions_by_array(pred_lgb_test, 'submission-lgb-trainval-all-adjusted-ext.csv')
# LB score 0.949 / 0.953
# ext: 0.943/0.


In [ ]:
!pip install catboost

In [ ]:
import catboost

from catboost import CatBoostRegressor, Pool

model=CatBoostRegressor(iterations=100)
model.fit(X_train, y_train, verbose=1)
pred_catboost_val = model.predict(X_val.values)

print('Validation R-squared for catboost is %f' % r2_score(y_val, pred_catboost_val))



In [ ]:
model=CatBoostRegressor(iterations=100)
model.fit(X_trainval, y_trainval, verbose=1)
pred_catboost_test = model.predict(X_test)
write_predictions_by_array(np.clip(pred_catboost_test,0,20), 'submission-catboost-trainval-ext.csv')

# LB score 0.986/0.996


In [ ]:
import lightgbm as lgb

lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }

model = lgb.train(lgb_params, lgb.Dataset(X_val, label=y_val), 100)
pred_lgb_test = np.clip(model.predict(X_test), 0, 20)
write_predictions_by_array(pred_lgb_test, 'submission-lgb-valonly.csv')
# LB score 1.10/1.10
